This notebook is used to check the migration of PBs method from SimaPro to Brightway2

For that, its performance will be checked against the dataset of chemicals from Green Chemistry paper.

# Read set of SimaPro solved chemicals

In [1]:
import re
from pathlib import Path

import brightway2 as bw
import pandas as pd
from project_path import ROOT_DIR
import project_path
from src.utils import progressbar, solve_lca

In [2]:
import bw2data

In [3]:
from bw_recipe_2016 import add_recipe_2016
import aesa_pbs
aesa_pbs.__version__

'0.8.6+splitCBI.0'

In [4]:
SIMAPRO_DATA = Path(ROOT_DIR + r"\data\raw\lcia-results-from-sp910-combined.xlsx")

In [5]:
with pd.ExcelFile(SIMAPRO_DATA, engine=None) as xl_obj:
    df_simapro = pd.read_excel(xl_obj, sheet_name="Sheet1")

In [6]:
df_simapro.head(3)

,wkbName,Activity,"('PBs-LCIA V0.71 V0.71', 'Climate change - CO2 concentration', 'ppm')","('PBs-LCIA V0.71 V0.71', 'Climate change - Energy imbalance', 'Wm-2')","('PBs-LCIA V0.71 V0.71', 'Stratospheric ozone depletion', 'DU')","('PBs-LCIA V0.71 V0.71', 'Ocean acidification', 'Omega Aragon')","('PBs-LCIA V0.71 V0.71', 'Biogeochemical flows - P', 'Tg P')","('PBs-LCIA V0.71 V0.71', 'Biogeochemical flows - N', 'Tg N')","('PBs-LCIA V0.71 V0.71', 'Land-system change - Global', '%')","('PBs-LCIA V0.71 V0.71', 'Freshwater use - Global', 'km3')",...,"('ReCiPe 2016 Endpoint (H) V1.03 / World (2010) H/A', 'Human health', 'DALY')","('ReCiPe 2016 Endpoint (H) V1.03 / World (2010) H/A', 'Ecosystems', 'species.yr')","('ReCiPe 2016 Endpoint (H) V1.03 / World (2010) H/A', 'Resources', 'USD2013')","('IPCC 2013 GWP 100a V1.03', 'IPCC GWP 100a', 'kg CO2 eq')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Non renewable, fossil', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Non-renewable, nuclear', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Non-renewable, biomass', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Renewable, biomass', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Renewable, wind, solar, geothe', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Renewable, water', 'MJ')"
0,Chemicals-Acids (inorganic)-Market,Sulfuric acid {RoW}| market for sulfuric acid ...,3.682300e-12,4.965498e-14,1.156094e-15,1.125213e-14,2.047673e-16,3.232508e-14,1.099348e-16,7.588386e-13,...,0.000002,2.389076e-09,0.052662,0.156735,6.316318,0.151773,0.000150,0.121871,0.011526,0.087806
1,Chemicals-Acids (inorganic)-Market,Sulfuric acid {RER}| market for sulfuric acid ...,2.682016e-12,3.621578e-14,8.425343e-16,8.195512e-15,2.162822e-16,3.982650e-14,1.233763e-16,2.679813e-13,...,0.000002,2.382083e-09,0.060525,0.119277,7.175078,0.233407,0.000138,0.073102,0.016586,0.064712
2,Chemicals-Acids (inorganic)-Market,Sulfamic acid {GLO}| market for sulfamic acid ...,5.439020e-11,7.271744e-13,8.913290e-15,1.662025e-13,1.122301e-15,2.008279e-13,5.993186e-16,4.956729e-12,...,0.000007,1.325358e-08,0.272812,2.172702,35.989036,2.025470,0.000812,0.479571,0.171391,0.812977


In [7]:
# df_simapro.columns

In [8]:
[i for i in df_simapro.columns if "ReCiPe 2016 Midpoint" in i]

["('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Stratospheric ozone depletion', 'kg CFC11 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ionizing radiation', 'kBq Co-60 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Human health', 'kg NOx eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Fine particulate matter formation', 'kg PM2.5 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Terrestrial ecosystems', 'kg NOx eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial acidification', 'kg SO2 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Freshwater eutrophication', 'kg P eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine eutrophication', 'kg N eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial ecotoxicity', 'kg 1,4-DCB')",
 "('ReCiPe

In [9]:
[i for i in df_simapro.columns if "0.72" in i]

["('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')",
 "('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')",
 "('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')",
 "('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')",
 "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')",
 "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - N', 'Tg N')",
 "('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')",
 "('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')",
 "('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')"]

```python
# all ReCiPe2016 categories implemented in bw_recipe_2016
# use them to update `map_methods`
[
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Fossil resource scarcity"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ozone Formation", "Damage to Ecosystems"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Marine eutrophication"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Terrestrial Acidification"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ozone Formation", "Damage to Humans"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "Infinite timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Land occupation"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Freshwater Eutrophication"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Water consumption"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Hierarchist", ),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Egalitarian", ),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Land transformation"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Egalitarian"),


("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "Infinite timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "100 year timescale","Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "1000 year timescale","Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine","Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Individualist"),
]
```

In [28]:
map_methods = {
    "Activity": "Activity",
    "('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'freshwater use', 'global'),
    "('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'climate change', 'atmospheric CO2 concentration'),
    "('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'change in biosphere integrity', 'functional diversity', 'total'),
    "('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'ocean acidification', 'carbonate ion concentration'),
    "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'biogeochemical flows', 'phosphorus'),
    "('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'climate change','energy imbalance at top-of-atmosphere'),
    "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - N', 'Tg N')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'biogeochemical flows', 'nitrogen'),
    "('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'ozone depletion', 'stratospheric O3 concentration'),
    "('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'land-system change', 'global'),
    "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq')": ('ReCiPe 2016', '1.1 (20180117)', 'Midpoint', 'Global Warming', '100 year timescale', 'Hierarchist'),
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Stratospheric ozone depletion', 'kg CFC11 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ionizing radiation', 'kBq Co-60 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Human health', 'kg NOx eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Fine particulate matter formation', 'kg PM2.5 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Terrestrial ecosystems', 'kg NOx eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial acidification', 'kg SO2 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Freshwater eutrophication', 'kg P eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine eutrophication', 'kg N eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial ecotoxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Freshwater ecotoxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine ecotoxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Human carcinogenic toxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Human non-carcinogenic toxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Land use', 'm2a crop eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Mineral resource scarcity', 'kg Cu eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Fossil resource scarcity', 'kg oil eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Water consumption', 'm3'): "
}

In [29]:
rev_map_methods = {v: k for k, v in map_methods.items()}

In [30]:
df_sp_selected = df_simapro.rename(columns=map_methods)[map_methods.values()]
df_sp_selected.head(3)

C:\Users\ViteksPC\VirtualEnvs\bw2\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


,Activity,"(AESA (PBs-LCIA), 0.8.6+splitCBI.0, freshwater use, global)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, climate change, atmospheric CO2 concentration)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, change in biosphere integrity, functional diversity, total)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, ocean acidification, carbonate ion concentration)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, biogeochemical flows, phosphorus)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, climate change, energy imbalance at top-of-atmosphere)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, biogeochemical flows, nitrogen)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, ozone depletion, stratospheric O3 concentration)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, land-system change, global)","(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist)"
0,Sulfuric acid {RoW}| market for sulfuric acid ...,7.588386e-13,3.687066e-12,4.280597e-14,1.126669e-14,2.047673e-16,4.971751e-14,3.232508e-14,1.156094e-15,1.099348e-16,0.160737
1,Sulfuric acid {RER}| market for sulfuric acid ...,2.679813e-13,2.685277e-12,3.340084e-14,8.205477e-15,2.162822e-16,3.625857e-14,3.982650e-14,8.425343e-16,1.233763e-16,0.123240
2,Sulfamic acid {GLO}| market for sulfamic acid ...,4.956729e-12,5.443214e-11,5.046367e-13,1.663306e-13,1.122301e-15,7.277248e-13,2.008279e-13,8.913290e-15,5.993186e-16,2.202389


In [31]:
len(list(df_sp_selected.Activity))

947

In [32]:
# list(df_simapro.Activity)

In [33]:
pattern = re.compile(r"(^.*) {(.*)}\| (.*) \|")

In [34]:
activities_sp = {}
for i in list(df_sp_selected.Activity):
    mo = re.match(pattern, i)
    if mo:
        ref_prod = mo[1].lower()
        location = mo[2]
        name = mo[3].lower()
        activities_sp[i] = {"reference product": ref_prod, "name": name, "location": location}
# activities_sp

# Prepare bw project

In [35]:
bw.projects

Brightway2 projects manager with 14 objects:
	StepByStep
	bw2_class_2020_example_database
	bw2_class_2020_intro
	car comparison paper
	check_PBsmethod
	check_impact_electricity
	default
	double_count_electricity
	ecoinvent 3.6
	global_power_mix
	parameters - manual creation
	premise_scenarios
	simapro-ecoinvent-import
	trial-import_from_SimaPro
Use `projects.report()` to get a report on all projects.

In [36]:
bw.projects.set_current("check_PBsmethod")

In [37]:
bw.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei35apos

In [38]:
bw.projects.dir

'C:\\Users\\ViteksPC\\AppData\\Local\\pylca\\Brightway3\\check_PBsmethod.fbe9669d91325ca0149ce34bf0d8ff48'

In [39]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


In [40]:
if "ei35apos" in bw.databases:
    print("Database already present! No import is needed.")
else:
    PATH_TO_EI_DATASETS = r"C:\Users\ViteksPC\Documents\00-ETH_projects\18-BW2Hub\01.Data\Ecoinvent\v3.5\ecoinvent 3.5_apos_ecoSpold02\datasets"
    ei = bw.SingleOutputEcospold2Importer(PATH_TO_EI_DATASETS, "ei35apos")
    # ei
    ei.apply_strategies()
    ei.statistics()

    # write database to disk --> actually saves it
    ei.write_database()
    del ei

Database already present! No import is needed.


In [41]:
bw.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei35apos

In [24]:
force_reinstall = True

if (
    any(
        "AESA" in str(aesa) and str(aesa_pbs.__version__) in str(aesa)
        for aesa in bw.methods
    )
    and not force_reinstall
):
    print("AESA methods already present!")
else:
    aesa_pbs.add_aesa_pbs(verbose=False)

Adding ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'climate change', 'atmospheric CO2 concentration')
Adding ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'ozone depletion', 'stratospheric O3 concentration')
Adding ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'land-system change', 'global')
Adding ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'change in biosphere integrity', 'functional diversity', 'direct land use')
Adding ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'ocean acidification', 'carbonate ion concentration')
Adding ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'biogeochemical flows', 'nitrogen')
Adding ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'biogeochemical flows', 'phosphorus')
Adding ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'change in biosphere integrity', 'functional diversity', 'total')
Adding ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'freshwater use', 'global')
Adding ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'climate change', 'energy imbalance at top-of-atmosphere')
Adding ('AESA (PBs-LCIA)', '0.8.6+spl

In [25]:
if any(rcp for rcp in bw.methods if "recipe 2016" in str(rcp).lower()):
    print("ReCiPe2016 methods already present!")
else:
    add_recipe_2016()

ReCiPe2016 methods already present!


In [42]:
recipe2016 = [rcp for rcp in bw.methods if "recipe 2016" in str(rcp).lower()]
pbs = [aesa for aesa in bw.methods if "AESA" in str(aesa) and str(aesa_pbs.__version__) in str(aesa)]

In [43]:
# for m in recipe2016:
for m in pbs:
    for k, v in rev_map_methods.items():
        if str(k) in str(m):
            print(v)

('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')
('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')
('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')
('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')
('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - N', 'Tg N')
('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')
('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')
('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')
('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')


In [44]:
eidb = bw.Database("ei35apos")
# eidb

In [45]:
trial = eidb.random()

In [46]:
trial.as_dict().keys()

dict_keys(['comment', 'classifications', 'activity type', 'activity', 'database', 'filename', 'location', 'name', 'synonyms', 'parameters', 'authors', 'type', 'reference product', 'flow', 'unit', 'production amount', 'code'])

In [47]:
trial["name"]

'treatment of biowaste by anaerobic digestion'

In [48]:
trial.as_dict()["name"]

'treatment of biowaste by anaerobic digestion'

In [49]:
trial.as_dict()["code"]

'e5f4c4b3cc6e4eadce289ac46b2cb95d'

In [50]:
trial.key

('ei35apos', 'e5f4c4b3cc6e4eadce289ac46b2cb95d')

In [51]:
# [i for i in trial.biosphere()]

In [52]:
# [i for i in trial.exchanges()]

In [53]:
[i for i in trial.production()]

[Exchange: -1.0 kilogram 'treatment of biowaste by anaerobic digestion' (kilogram, RoW, None) to 'treatment of biowaste by anaerobic digestion' (kilogram, RoW, None)>]

In [54]:
# [i for i in trial.substitution()]
# [i for i in trial.technosphere()]

In [55]:
# [i for i in trial.upstream()]

In [56]:
activities_sp['Sulfuric acid {RoW}| market for sulfuric acid | APOS, S'].keys()

dict_keys(['reference product', 'name', 'location'])

In [57]:
eidb_markets = [activity for activity in eidb if "market" in activity["name"]]

activities_bw = []

for act_sp in progressbar(
    itobj=activities_sp.keys(),
    prefix="Progress: ",
    size=20,
    unit="activity(ies)",
):
    # activities_sp.setdefault("spname", None)
    for act in eidb_markets:
        if (
            activities_sp[act_sp]["reference product"].lower() == str(act["reference product"]).lower()
            and "market" in str(act["name"])
            and activities_sp[act_sp]["location"] == str(act["location"])
        ):
            activities_bw.append(act)
            activities_sp[act_sp].update({"einame": act})


Progress: [🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩] 947/947 activity(ies)
Duration: 0:00:09.748346


In [58]:
[i for i, a in activities_sp.items() if "market" in a.get("einame", "nope")]

[]

In [59]:
# len(
#     set(activities_bw)- set([b["spname"] for b in activities_sp.values() if b.get("spname")])
# )

# activities not returned to activities_sp!!!
# STOPPED HERE!
set(activities_bw)- set([b["einame"] for b in activities_sp.values() if b.get("einame")])

{'market for neutralising agent, sodium hydroxide-equivalent' (kilogram, GLO, None),
 'fraction 1 from naphtha separation to generic market for chemical, organic' (kilogram, GLO, None),
 'methylcyclopentane to generic market for solvent, organic' (kilogram, GLO, None),
 '1,1-dimethylcyclopentane to generic market for solvent, organic' (kilogram, GLO, None),
 'white spirit to generic market for solvent for paint' (kilogram, GLO, None),
 'praseodymium oxide to generic market for mischmetal' (kilogram, GLO, None),
 'acetonitrile to generic market for solvent for chromatography analysis' (kilogram, GLO, None),
 'market for vegetable oil, refined' (kilogram, GLO, None),
 'market for soil pH raising agent, as CaCO3' (kilogram, GLO, None),
 'ethanol, from fermentation, to niche market for ethanol, at service station' (kilogram, CH, None),
 'market for solvent for chromatography analysis' (kilogram, GLO, None),
 'palm oil, refined, to generic market for vegetable oil' (kilogram, GLO, None),
 '

In [60]:
len(activities_bw)

978

In [61]:
978-33

945

In [62]:
len(set(activities_bw))

978

In [63]:
871+76

947

In [64]:
len(activities_sp)

947

In [65]:
[i for i in activities_sp if activities_sp[i].get("einame", None)==None]

['Deinking emulsion, in paper production {GLO}| market for | APOS, S',
 'Rare earth concentrate, 70% REO, from bastnasite {GLO}| market for | APOS, U - copy']

In [66]:
activities_sp['Sulfuric acid {RoW}| market for sulfuric acid | APOS, S']["einame"]

'market for sulfuric acid' (kilogram, RoW, None)

In [67]:
def get_einame(key):
    return activities_sp[key].get("einame", None)#["activity"]

In [68]:
get_einame('Sulfuric acid {RoW}| market for sulfuric acid | APOS, S')

'market for sulfuric acid' (kilogram, RoW, None)

In [69]:
df_sp_selected['einame'] = df_sp_selected['Activity'].apply(lambda key: get_einame(key))
df_sp_selected

,Activity,"(AESA (PBs-LCIA), 0.8.6+splitCBI.0, freshwater use, global)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, climate change, atmospheric CO2 concentration)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, change in biosphere integrity, functional diversity, total)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, ocean acidification, carbonate ion concentration)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, biogeochemical flows, phosphorus)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, climate change, energy imbalance at top-of-atmosphere)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, biogeochemical flows, nitrogen)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, ozone depletion, stratospheric O3 concentration)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, land-system change, global)","(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist)",einame
0,Sulfuric acid {RoW}| market for sulfuric acid ...,7.588386e-13,3.687066e-12,4.280597e-14,1.126669e-14,2.047673e-16,4.971751e-14,3.232508e-14,1.156094e-15,1.099348e-16,0.160737,"[comment, classifications, activity type, acti..."
1,Sulfuric acid {RER}| market for sulfuric acid ...,2.679813e-13,2.685277e-12,3.340084e-14,8.205477e-15,2.162822e-16,3.625857e-14,3.982650e-14,8.425343e-16,1.233763e-16,0.123240,"[comment, classifications, activity type, acti..."
2,Sulfamic acid {GLO}| market for sulfamic acid ...,4.956729e-12,5.443214e-11,5.046367e-13,1.663306e-13,1.122301e-15,7.277248e-13,2.008279e-13,8.913290e-15,5.993186e-16,2.202389,"[comment, classifications, activity type, acti..."
3,"Phosphoric acid, industrial grade, without wat...",2.076097e-11,3.683377e-11,7.170159e-13,1.125543e-13,5.740347e-14,4.924900e-13,4.259820e-13,6.920908e-15,8.400603e-16,1.491608,"[comment, classifications, activity type, acti..."
4,"Phosphoric acid, fertiliser grade, without wat...",1.415177e-11,2.322850e-11,5.772262e-13,7.098025e-14,5.402468e-14,3.118323e-13,3.214601e-13,4.997489e-15,8.019743e-16,0.957087,"[comment, classifications, activity type, acti..."
...,...,...,...,...,...,...,...,...,...,...,...,...
942,"Polyvinylchloride, suspension polymerised {GLO...",4.007584e-12,5.098499e-11,4.271375e-13,1.557991e-13,4.873162e-15,6.738817e-13,1.941198e-13,7.519967e-15,3.080851e-17,2.035933,"[comment, classifications, activity type, acti..."
943,"Polyvinylidenchloride, granulate {RER}| market...",1.261049e-12,1.167023e-10,1.113848e-12,3.566084e-13,6.547821e-15,1.545165e-12,1.637795e-13,1.837176e-15,-2.863068e-18,5.330478,"[comment, classifications, activity type, acti..."
944,"Polyvinylidenchloride, granulate {RoW}| market...",1.275448e-12,1.183721e-10,1.133030e-12,3.617105e-13,6.524817e-15,1.566875e-12,1.181972e-13,1.901179e-15,1.994662e-17,5.399748,"[comment, classifications, activity type, acti..."
945,"Tetrafluoroethylene {GLO}| market for | APOS, U",2.032338e-11,1.900741e-10,2.871105e-11,5.808169e-13,9.836993e-15,6.185484e-12,2.573934e-12,3.205619e-11,1.585247e-15,137.134320,"[comment, classifications, activity type, acti..."


In [70]:
df_sp_selected = df_sp_selected.iloc[[i for i in df_sp_selected.index if df_sp_selected.einame[i] is not None]]

In [71]:
# type(df_sp_selected.einame[665])

In [72]:
# type(activities_bw[0])

In [73]:
df_sp_selected.shape

(945, 12)

In [74]:
c = 0
for idx in df_sp_selected.index:
    if not isinstance(df_sp_selected.einame[idx], bw2data.backends.peewee.proxies.Activity):
        print(df_sp_selected.Activity[idx])
        print(idx)
        c += 1
c

0

In [75]:
dct_FUs = [{act: 1} for act in df_sp_selected.einame] # if isinstance(act, bw2data.backends.peewee.proxies.Activity)]
# dct_FUs = [{act: 1} for act in activities_bw]
# dct_FUs

In [76]:
# FU0 = dct_FUs[45]
# all_scores = {}
# lca = bw.LCA(FU0, pbs[0])
# lca.lci()
# lca.lcia()
# for category in pbs:
#     lca.switch_method(category)
#     lca.lcia()
#     all_scores[category] = {}
#     all_scores[category]["score"] = lca.score
#     all_scores[category]["unit"] = bw.Method(category).metadata["unit"]

In [77]:
# FU0

In [78]:
pd.options.display.float_format = '{:12.5e}'.format

In [79]:
# df_temp = pd.DataFrame.from_dict(all_scores).T
# df_temp

In [80]:
bw.calculation_setups

CalculationSetups dictionary with 3 object(s):
	chemicals
	n
	other

In [81]:
pbs + ["'ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq'"]

[('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'climate change',
  'atmospheric CO2 concentration'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'ozone depletion',
  'stratospheric O3 concentration'),
 ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'land-system change', 'global'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'change in biosphere integrity',
  'functional diversity',
  'direct land use'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'ocean acidification',
  'carbonate ion concentration'),
 ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'biogeochemical flows', 'nitrogen'),
 ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'biogeochemical flows', 'phosphorus'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'change in biosphere integrity',
  'functional diversity',
  'total'),
 ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'freshwater use', 'global'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'climate change',
  'energy imbalance at top-of-atmosphere'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'c

In [82]:
selected_methods = pbs + [
    m for k, v in rev_map_methods.items() for m in recipe2016 if str(k) in str(m)
]

In [83]:
bw.calculation_setups["chemicals"] = {
    "inv": dct_FUs,
    "ia": selected_methods,
}

In [84]:
# [act["name"] for FU in myMultiLCA.func_units for act in FU.keys()]

In [85]:
# bw.calculation_setups["chemicals"]
myMultiLCA = bw.MultiLCA("chemicals")
# myMultiLCA.results

df_impact = pd.DataFrame(
    data=myMultiLCA.results,
    columns=selected_methods,
    # index=[[act for FU in myMultiLCA.func_units for act in FU.keys()]]
    index=[
        tuple([act["name"], act["location"], act["reference product"]])
        for FU in myMultiLCA.func_units
        for act in FU
    ],
)
df_impact.head(3)

,"(AESA (PBs-LCIA), 0.8.6+splitCBI.0, climate change, atmospheric CO2 concentration)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, ozone depletion, stratospheric O3 concentration)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, land-system change, global)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, change in biosphere integrity, functional diversity, direct land use)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, ocean acidification, carbonate ion concentration)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, biogeochemical flows, nitrogen)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, biogeochemical flows, phosphorus)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, change in biosphere integrity, functional diversity, total)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, freshwater use, global)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, climate change, energy imbalance at top-of-atmosphere)","(AESA (PBs-LCIA), 0.8.6+splitCBI.0, change in biosphere integrity, functional diversity, CO2eq emissions)","(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist)"
"(market for sulfuric acid, RoW, sulfuric acid)",3.68707e-12,1.15609e-15,1.09935e-16,1.25277e-14,1.12667e-14,4.67079e-14,2.04767e-16,4.59113e-14,2.02947e-11,4.97176e-14,3.33836e-14,1.60736e-01
"(market for sulfuric acid, RER, sulfuric acid)",2.68528e-12,8.42535e-16,1.23376e-16,1.06638e-14,8.20548e-15,4.97176e-14,2.16282e-16,3.62595e-14,2.59210e-11,3.62586e-14,2.55958e-14,1.23239e-01
"(market for sulfamic acid, GLO, sulfamic acid)",5.44322e-11,8.91329e-15,5.99318e-16,6.40927e-14,1.66331e-13,1.29056e-12,1.12230e-15,5.21511e-13,9.56326e-11,7.27725e-13,4.57418e-13,2.20238e+00


In [86]:
df_impact.shape

(945, 12)

In [87]:
# [(act["name"], act["location"]) for FU in myMultiLCA.func_units for act in FU]

In [88]:
indices = [tuple([act["name"], act["location"], act["reference product"]]) for act in df_sp_selected.einame]

In [89]:
# df_sp_selected

In [90]:
df_sp_selected["einame_idx"] = indices

C:\Users\ViteksPC\AppData\Local\Temp/ipykernel_28604/218076019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sp_selected["einame_idx"] = indices


In [91]:
df_sp_selected.set_index("einame_idx", inplace=True)
df_sp_selected.drop("Activity", 1, inplace=True)
df_sp_selected.drop("einame", 1, inplace=True)

C:\Users\ViteksPC\AppData\Local\Temp/ipykernel_28604/3607502913.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_sp_selected.drop("Activity", 1, inplace=True)
C:\Users\ViteksPC\AppData\Local\Temp/ipykernel_28604/3607502913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sp_selected.drop("Activity", 1, inplace=True)
C:\Users\ViteksPC\AppData\Local\Temp/ipykernel_28604/3607502913.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_sp_selected.drop("einame", 1, inplace=True)
C:\Users\ViteksPC\AppData\Local\Temp/ipykernel_28604/3607502913.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [92]:
df_sp_selected.shape

(945, 10)

In [93]:
import numpy as np

In [94]:
# np.isclose(7.40056E-13*10e13,1.05827e-13*10e13)

In [95]:
df_impact.shape

(945, 12)

# one-by-one comparisons

In [96]:
def reindex_pbs_scores(data: pd.DataFrame) -> pd.DataFrame:
    to_return = data.reindex(
        columns=[
            "climate change - atmospheric CO2 concentration",
            "climate change - energy imbalance at top-of-atmosphere",
            "ozone depletion - stratospheric O3 concentration",
            "ocean acidification - carbonate ion concentration",
            "biogeochemical flows - phosphorus",
            "biogeochemical flows - nitrogen",
            "land-system change - global",
            "freshwater use - global",
            "change in biosphere integrity - functional diversity",
        ],
        level=0,
    ).T
    return to_return

In [97]:
# axt = [
#     a
#     for a in eidb
#     if "heat and power co-generation, wood chips, 6667 kW, state" in a["name"]
#     and "electricity" in a["reference product"]
#     and "ES" == a["location"]
#     and "label-certified" not in a["name"]
# ]  # , 'green manure, Swiss integrated production, until March')

# axt = [a for a in eidb if "market for sulfuric acid" in a["name"] and "RER" in a["location"]]

# axt = [a for a in eidb if "market for silicon, multi-Si, casted" in a["name"] and "CA-QC" in a["location"]]

# axt = [a for a in eidb if "lime to generic market for soil pH raising agent" in a["name"] and "GLO" in a["location"]]
axt = [a for a in eidb if "market for potassium fertiliser, as K2O" in a["name"] and "GLO" in a["location"]]

# axt = [
#     a
#     for a in eidb
#     if "market for green manure, Swiss integrated production, until March" in a["name"]
#     and "GLO" in a["location"]
# ]

axt

['market for potassium fertiliser, as K2O' (kilogram, GLO, None)]

In [98]:
results_df = reindex_pbs_scores(solve_lca(axt, pbs))
results_df

,Name,"market for potassium fertiliser, as K2O"
,Location,GLO
Method,Unit,
climate change - atmospheric CO2 concentration,ppm,3.86726e-11
climate change - energy imbalance at top-of-atmosphere,Wm-2,5.83656e-13
ozone depletion - stratospheric O3 concentration,Dobson unit,2.19451e-13
ocean acidification - carbonate ion concentration,omega aragonite,1.18167e-13
biogeochemical flows - phosphorus,Tg P,1.04999e-13
biogeochemical flows - nitrogen,Tg N,3.01576e-11
land-system change - global,% forested land,1.27769e-14
freshwater use - global,km3,1.37133e-10


In [99]:
sp_results = {
    "climate change - atmospheric CO2 concentration": 3.8672584E-11,
    "climate change - energy imbalance at top-of-atmosphere": 5.8365615E-13,
    "ozone depletion - stratospheric O3 concentration":2.1945121E-13,
    "ocean acidification - carbonate ion concentration": 1.1816742E-13,
    "biogeochemical flows - phosphorus": 1.0499918E-13,
    "biogeochemical flows - nitrogen": 3.0157583E-11,
    "land-system change - global": 1.2776855E-14,
    "freshwater use - global": 1.370915E-10,
    "change in biosphere integrity - functional diversity": 1.6649877E-12,
}
sp_results_df = pd.DataFrame().from_dict(
    sp_results,
    orient="index",
    columns=[results_df.columns[0][0]],
)
sp_results_df

,"market for potassium fertiliser, as K2O"
climate change - atmospheric CO2 concentration,3.86726e-11
climate change - energy imbalance at top-of-atmosphere,5.83656e-13
ozone depletion - stratospheric O3 concentration,2.19451e-13
ocean acidification - carbonate ion concentration,1.18167e-13
biogeochemical flows - phosphorus,1.04999e-13
biogeochemical flows - nitrogen,3.01576e-11
land-system change - global,1.27769e-14
freshwater use - global,1.37092e-10
change in biosphere integrity - functional diversity,1.66499e-12


In [100]:
# relative difference of bw vs sp results
# negative - in bw is less than in sp
# positive - in bw is more than in sp

bw_res = results_df.droplevel(1, axis=0).droplevel(1, axis=1)
sp_res = sp_results_df
rel_diff = (bw_res-sp_res)/sp_res*100
rel_diff.iloc[:,0] = rel_diff.iloc[:,0].map('{:.2f}%'.format)
rel_diff

Name,"market for potassium fertiliser, as K2O"
Method,
climate change - atmospheric CO2 concentration,-0.00%
climate change - energy imbalance at top-of-atmosphere,-0.00%
ozone depletion - stratospheric O3 concentration,0.00%
ocean acidification - carbonate ion concentration,-0.00%
biogeochemical flows - phosphorus,0.00%
biogeochemical flows - nitrogen,0.00%
land-system change - global,-0.00%
freshwater use - global,0.03%
change in biosphere integrity - functional diversity,-30.75%


In [101]:
from collections import defaultdict


def top_emissions_by_name(lca, biosphere_database="biosphere3"):
    names = defaultdict(list)

    for flow in bw.Database(biosphere_database):
        if flow.key in lca.biosphere_dict:
            names[flow["name"]].append(
                lca.characterized_inventory[lca.biosphere_dict[flow.key], :].sum()
            )

    return sorted([(sum(scores), name) for name, scores in names.items()], reverse=True)

In [102]:
pbs

[('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'climate change',
  'atmospheric CO2 concentration'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'ozone depletion',
  'stratospheric O3 concentration'),
 ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'land-system change', 'global'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'change in biosphere integrity',
  'functional diversity',
  'direct land use'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'ocean acidification',
  'carbonate ion concentration'),
 ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'biogeochemical flows', 'nitrogen'),
 ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'biogeochemical flows', 'phosphorus'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'change in biosphere integrity',
  'functional diversity',
  'total'),
 ('AESA (PBs-LCIA)', '0.8.6+splitCBI.0', 'freshwater use', 'global'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'climate change',
  'energy imbalance at top-of-atmosphere'),
 ('AESA (PBs-LCIA)',
  '0.8.6+splitCBI.0',
  'c

In [103]:
lca = bw.LCA({axt[0]: 1}, pbs[1])
lca.lci()
lca.lcia()
lca.score

2.1945121955073056e-13

In [104]:
lca.demand

{'market for potassium fertiliser, as K2O' (kilogram, GLO, None): 1}

In [105]:
(7.010726210309949e-14-4.38e-14)/4.38e-14*(7.010726210309949e-14/lca.score)

0.19187860373512142

In [106]:
5.522868070172049e-16/lca.score

0.0025166723071663438

In [107]:
top_emissions_by_name(lca)[:50]

[(2.1852461552502545e-13, 'Dinitrogen monoxide'),
 (6.79470419898571e-16, 'Methane, bromotrifluoro-, Halon 1301'),
 (9.665282489227307e-17, 'Methane, tetrachloro-, R-10'),
 (8.427477639471253e-17, 'Methane, bromochlorodifluoro-, Halon 1211'),
 (2.289048253430016e-17, 'Ethane, 1,2-dichloro-1,1,2,2-tetrafluoro-, CFC-114'),
 (2.157610489719575e-17, 'Methane, dichlorodifluoro-, CFC-12'),
 (1.3371430741657356e-17, 'Methane, chlorodifluoro-, HCFC-22'),
 (3.929831250817409e-18, 'Methane, monochloro-, R-40'),
 (3.3929639811163773e-18, 'Ethane, 1,1,2-trichloro-1,2,2-trifluoro-, CFC-113'),
 (5.650772674995467e-19, 'Methane, bromo-, Halon 1001'),
 (4.684695251759909e-19, 'Ethane, 1,1,1-trichloro-, HCFC-140'),
 (1.1644321788276503e-20, 'Methane, trichlorofluoro-, CFC-11'),
 (0.0, 'tau-Fluvalinate'),
 (0.0, 't-Butylamine'),
 (0.0, 't-Butyl methyl ether'),
 (0.0, 'o-Xylene'),
 (0.0, 'o-Nitrotoluene'),
 (0.0, 'o-Dichlorobenzene'),
 (0.0, 'm-Xylene'),
 (0.0, '[Deleted]Tri-allate'),
 (0.0, '[Deleted]Ca

In [108]:
# contributions in sp can be found in the "Inventory" tab

# for Electricity, high voltage {CH}| heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014 | APOS, U
# that yields the same score in CBI
# the main contributor is "Occupation, forest, extensive" ~94% of impact

# on the other hand, for 'market for sulfuric acid' (kilogram, RoW, None) the ~65% of impact is 
# of "Carbon dioxide, fossil" in sp, BUT NOT IN bw!!!
# so the problem is the ReCiPe method???
# in bw "Carbon dioxide, fossil" yields 8.2e-15
# while in sp - 2.78e-14

In [109]:
l = [t[0] for t in top_emissions_by_name(lca)[1:]]
print(sum(l))
# np.isclose(sum(l), lca.score)
1-sum(l)/lca.score

9.266040257051077e-16


0.9957776310033634

In [110]:
(8.281146767077711e-15+8.237074606492378e-15+1.8354772572644123e-15+1.3654838175278122e-15)/lca.score

0.08985679135769772

In [111]:
2e-14 == 0.2e-13

True

In [112]:
import bw2analyzer as bwa

In [113]:
# see impact contributors to an activity group by CPC classification
# if "relative" - shows fractional contribution to total

# bwa.compare_activities_by_grouped_leaves(axt, pbs[2], mode="relative", output_format="list", max_level=5)

In [114]:
ca = bwa.ContributionAnalysis()
# ca.annotated_top_processes(lca, limit=5)
ca.annotated_top_emissions(lca, limit=0.00001, limit_type='percent')

[(1.424993121251766e-13,
  0.0010106333953896764,
  'Dinitrogen monoxide' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (7.481816899652522e-14,
  0.0005306252994637264,
  'Dinitrogen monoxide' (kilogram, None, ('air', 'urban air close to ground'))),
 (1.2047722487849545e-15,
  8.544483831551588e-06,
  'Dinitrogen monoxide' (kilogram, None, ('air',))),
 (6.793851093927694e-16,
  5.907696662132669e-09,
  'Methane, bromotrifluoro-, Halon 1301' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (9.663970425504966e-17,
  1.6836185319828046e-08,
  'Methane, tetrachloro-, R-10' (kilogram, None, ('air', 'urban air close to ground'))),
 (8.42747763947125e-17,
  1.6332320779882228e-09,
  'Methane, bromochlorodifluoro-, Halon 1211' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (2.2890482534300168e-17,
  6.288593966121307e-09,
  'Ethane, 1,2-dichloro-1,1,2,2-tetrafluoro-, CFC-114' (kilogram, None, ('air', 'non-urban air or from high stacks')))

In [115]:
-4.373743127925108e-11+1.1730336203662644e-11+1.1320238491218266e-11+8.922289063820765e-12+6.830495504523874e-12

-4.934072016025528e-12

# results

In [116]:
res = (df_impact-df_sp_selected).div(df_sp_selected).mul(100)

In [117]:
# res

In [118]:
def make_pretty(styler):
    styler.background_gradient(axis=None, vmin=-100, vmax=100, cmap="seismic")
    return styler

In [119]:
# big diffrences in: 
# "market for green manure" due to diffenent units, bw -> ha, simapro -> m2
# lime to generic market for soil pH raising agent: recalculated:OK
# methyl acetate to generic market for solvent, organic', 'GLO: recalculated: OK

In [120]:
res.style.pipe(make_pretty)